In [7]:
import equivariant

eps = 1e-4

In [8]:
# Tests if all linear equivariant transformations are correct
# equivariant.test_all(eps)

In [9]:
# Tests the permutation equivariance/invariance of the layers
from layers import LinEq2v2, Msg, LinEq2v0
from keras.models import Sequential
from keras.layers import Dense, Flatten
import numpy as np
import tensorflow as tf


num_tests = 10


In [10]:
# Check perm invariance

model_perminv = Sequential(
    layers=
    [
        Msg(30, activation='sigmoid'),
        LinEq2v0(20, activation='sigmoid'),
        Dense(5)
    ]
)

N = np.random.randint(10, 20) # Number of particles

momentum = np.random.normal(
    size=(N, 3)
)

input_tensor = np.einsum("pi, qj->pq", momentum, momentum)
output = model_perminv(input_tensor.reshape((1, N, N, 1)))

for n in range(num_tests):
    perm = np.random.permutation(N)
    model_input = np.einsum("pi, qj->pq", momentum[perm], momentum[perm])
    model_input = np.reshape(model_input, (1, N, N, 1))
    model_output = model_perminv(model_input)
    diff = np.max(np.abs(output - model_output))

    assert(diff < eps)

print("All tests passed")


All tests passed


In [15]:
# Check perm equivariance (Know from test above that test is only invariant iff all layers are equivariant/invariant)

model_permeq = Sequential(
    layers=
    [
        Msg(30, activation='leaky_relu'),
        LinEq2v2(40, activation='leaky_relu'),

        LinEq2v0(20, activation='leaky_relu'),
        Dense(5, activation='softmax')
    ]
)

N = np.random.randint(10, 20) # Number of particles

momentum = np.random.normal(
    size=(N, 3)
)

input_tensor = np.einsum("pi, qj->pq", momentum, momentum)
output = model_permeq(input_tensor.reshape((1, N, N, 1)))

for n in range(num_tests):
    perm = np.random.permutation(N)
    model_input = np.einsum("pi, qj->pq", momentum[perm], momentum[perm])
    model_input = np.reshape(model_input, (1, N, N, 1))

    model_output = model_permeq(model_input)
    diff = np.max(np.abs(output - model_output))
    print(model_output, model_input[0][0][0][0])

    assert(diff < eps)

print("All tests passed")


tf.Tensor([[0. 0. 0. 1. 0.]], shape=(1, 5), dtype=float32) 14.42801085342983
tf.Tensor([[0. 0. 0. 1. 0.]], shape=(1, 5), dtype=float32) 0.021251503398170956
tf.Tensor([[0. 0. 0. 1. 0.]], shape=(1, 5), dtype=float32) 2.516883761611963
tf.Tensor([[0. 0. 0. 1. 0.]], shape=(1, 5), dtype=float32) 0.2520046524777635
tf.Tensor([[0. 0. 0. 1. 0.]], shape=(1, 5), dtype=float32) 15.409759274558818
tf.Tensor([[0. 0. 0. 1. 0.]], shape=(1, 5), dtype=float32) 0.10411007980244236
tf.Tensor([[0. 0. 0. 1. 0.]], shape=(1, 5), dtype=float32) 0.021251503398170956
tf.Tensor([[0. 0. 0. 1. 0.]], shape=(1, 5), dtype=float32) 23.470559799958544
tf.Tensor([[0. 0. 0. 1. 0.]], shape=(1, 5), dtype=float32) 0.005234974317019601
tf.Tensor([[0. 0. 0. 1. 0.]], shape=(1, 5), dtype=float32) 0.16261112410761153
All tests passed
